此文建立在最原始的linear GNN已经理解的基础，讲解。
参考资料：https://zhuanlan.zhihu.com/p/38612863

首先，GCN相对于GNN哪些地方变了？为什么变了？

那么首先我们来看GNN的全过程： 
$$X = F(X,l) \quad (0)$$
$$X = EH \quad (1)$$
$$O = G(X,l) \quad (2)$$
![avatar](equation1.jpg)
![avatar](equation2.jpg)
![avatar](equation3.jpg)

下面是关于这些公式的解释:  
首先是参数解释：  
不带下标表示总的，带下标表示某个。  
&emsp;X：节点的状态向量  
&emsp;l：节点或边的特征向量  
&emsp;h：更新过程中每条边产生的，用来更新状态向量的，堆叠而成的  
其它的变量可以参考论文，它们在这里不重要  

接下来是这几个方程的解释：  
首先(0),(2)是GNN最概括的表述，F函数用于更新，更新T次到达不动点后用G函数输出。那么这两个函数是怎么样的呢？  
先说（2），（2)比较容易实现，(2)直接根据你的任务用最简单的方式实现就行了，比如直接一个前馈式神经网络用一个`nn.Linear`，把X放进去吐出来就行了，当然，也可以把l也放进去，拼接一下或者分别用`nn.Linear`把维度弄相同叠加也可以  
然后（0），（0）的处理即是（3），把每条边用H函数来处理，然后更新，h函数的具体实现参考论文即可了，或者参考前面我的gnn的讲解  
但是那么方程（1）呢？这就是网上很多讲解GCN和GNN没有提及的。这是代码和论文中没有提及的地方。方程（3）如何代码实现呢？for循环一个个访问它的邻接点吗？本身这种方法也可行，但是很多实现方法不是这样的，而是：我们把所有边的H叠到一起，假设总共e条边，然后我们把每条边需要更新的节点也列成一个列表，长度为e，然后假设n个点，我这个列表复制n份叠成矩阵，然后对于拿去更新i节点的那一行就通通减去i,然后令这一行中=0的统统为1，其它为0，就得到了那个方程（3）中的E矩阵，然后用E\*H就更新了，这里的矩阵乘法就是这样来的,这种思想，在接下来的GCN也有所体现

那么现在GCN动了GNN的哪些地方呢？
其实改动特别大，主要动的便是方程（12）的实现。首先我们先假设方程（12）为
$$h_u=X_uW$$
那么我们可以发现，结合GNN用矩阵来代替for的思路，我们可以得到：
$$X^{t+1}=AX^{t}W$$
其中A为临接矩阵，可以XW就可以得到所有节点产生的用于更新的信息，再乘A就可以了。  
但是这样有问题。因为对于任一一个节点来说，本身的信息是很重要的，这个居然完全放弃了本身t时刻的状态向量。  
所以令A=A+I（I为单位矩阵），这样就补上了。
但是还有一个问题：A没有被规范化，这样是有问题的，比如相邻节点多的更新就明显比其它的大。  
那么问题就在于如何规范化A

这个时候就要引入卷积了。直接来看推导的结果（推导过程可以自行参考[网上的教程](http://xtf615.com/2019/02/24/gcn/)）这里仅记录理解的几个关键点：
![avatar](gcn.jpg)

其中H表示状态向量，其中：
$$\tilde{A} = A + I$$
$$\tilde{D} = D + I$$
其中D为度数矩阵，即对角线上点i的度数，其它全为0

公式推导的几个关键点：  
1.由傅里叶级数到傅里叶变换时，其实纯数学的方法更好理解，直接在级数的公式上代入an,bn,sin,cos(前两个就是用f(x)来表示的积分式子，后两个就是欧拉公式用e来表示的式子，即可得到cn的一个表达式，在那个式子另y=n/T,然后T取无穷很简单，一下就可以想明白。  
2.拉普拉斯算子那个教程讲的很详细了，离散的数据就是用近似的方法，可以结合导数定义，这样就好理解了。（本地名为laplace的截图就是这个）该博客中的D为度数矩阵，W为邻接矩阵，不带下标的f为所有点的值叠加起来的向量（矩阵）  
3.线性代数知识复习：  
半正定矩阵：设A是n阶实对称矩阵，如果对任何非零向量X，都有X'AX≥0，其中X'表示X的转置，就称A为半正定矩阵，正定就是是>0.    
特征值与特征向量：设 Ａ 是 ｎ 阶矩阵，如果数 λ 和 ｎ 维非零列向量 α 使关系式Ａα＝λα成立，那么数 λ 称为矩阵 Ａ 的特征值，非零向量 α 称为 Ａ 的对应于特征值 λ 的特征向量  
4.在类比时，特征函数就是由特征向量构成的函数，其任取一个值都可以作特征向量  
5.先依次看推导，当不懂时，往下看，看它是要干什么，再反过来看。比如看不懂图上的傅里叶变换了，就往下看看为什么要用这个东西，发现是类比CNN为了求卷积。  
6.现在学的GCN是第三代GCN，它的推导不是我上面说的那样，不过感觉可以这样理解。但是在理解为什么有卷积的时候，不要这样想，因为卷积并不是仅仅涉及ne[n],三代GCN的这个公式是简化来的

In [80]:
import os
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F

device = 'cpu'
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [81]:
'''
node_num, feat_dim, stat_dim, num_class
feat_Matrix, cites
'''
#数据处理
content_path = "./cora/cora.content"
cite_path = "./cora/cora.cites"

#读取文本内容
with open(content_path, "r") as fp:
    contents = fp.readlines()
with open(cite_path, "r") as fp:
    cites = fp.readlines()
    
contents = np.array([np.array(s.strip().split("\t")) for s in contents])
paper_list, feat_list, label_list = np.split(contents, [1,-1], axis=1)
paper_list, label_list = np.squeeze(paper_list), np.squeeze(label_list)

# paper -> index dict
#print("paper_list",sorted(paper_list))
paper_dict = dict([(key, val) for val, key in enumerate(paper_list)])

# lable -> index dict
labels = list(set(label_list))
label_dict = dict([(key, val) for val, key in enumerate(labels)])

# edge_index
cites = [i.strip().split("\t") for i in cites]
cites = np.array([[paper_dict[i[0]], paper_dict[i[1]]] for i in cites], dtype = np.int64)
node_num = len(paper_list)
feat_dim = feat_list.shape[1]
stat_dim = 128
num_class = len(labels)
feat_Matrix = torch.Tensor(feat_list.astype(np.float32))
label_list = np.array([label_dict[i] for i in label_list])
label_list = torch.tensor(label_list, dtype = torch.long)

In [82]:
print("Number of node : ", node_num)
print("Number of edges : ", cites.shape[0])
print("Number of classes : ", num_class)
print("Dimension of node features : ", feat_dim)
print("Dimension of node state : ", stat_dim)
print("Shape of feat_Matrix : ", feat_Matrix.shape)

Number of node :  2708
Number of edges :  5429
Number of classes :  7
Dimension of node features :  1433
Dimension of node state :  128
Shape of feat_Matrix :  torch.Size([2708, 1433])


In [83]:
class GCNlayer(nn.Module):
    def __init__(self, node_num, feat_num):
        super(GCNlayer, self).__init__()
        
        self.linear = nn.Linear(feat_num,feat_num,bias=False)
    
    def forward(self, matDAD, H):
        return F.relu(self.linear(torch.matmul(matDAD,H)))

In [84]:
class GCN(nn.Module):
    def __init__(self, node_num, feat_num, stat_num, num_class):
        super(GCN,self).__init__()
        
        self.input_layer = nn.Linear(feat_num, stat_num)
        self.gcn_layer1 = GCNlayer(node_num, stat_num)
        self.gcn_layer2 = GCNlayer(node_num, stat_num)
        self.out_layer = nn.Linear(stat_num, num_class)
        #这里天坑，由于不需要学习的我一向喜欢直接用F.而不是类
        #结果之前一直准确率不对，才发现如果用的是类在预测时会自动关闭的，而函数则不会
        self.dropout = nn.Dropout()
    
    def forward(self, matDAD, X):
        H = self.input_layer(X)
        H = self.gcn_layer1(matDAD, H)
        H = self.gcn_layer2(matDAD, H)
        output = F.log_softmax(self.dropout(self.out_layer(H)),dim = -1)
        return output

In [85]:
model1 = GCN(5,7,3,7)
matDAD1 = torch.randn(5,5)
H1 = torch.randn(5,7)
print(model1(matDAD1, H1))

tensor([[-2.0395, -2.0395, -2.0395, -2.0395, -2.0027, -1.5392, -2.0395],
        [-1.9801, -1.4591, -1.9801, -1.9801, -1.9801, -1.9801, -2.5606],
        [-2.5477, -1.7843, -1.7843, -1.7843, -1.8548, -1.7843, -2.3689],
        [-5.0380, -1.0789, -3.8959, -1.6337, -1.6337, -3.0446, -1.6337],
        [-2.0685, -1.5571, -2.4845, -1.3874, -2.0685, -2.0685, -2.5634]],
       grad_fn=<LogSoftmaxBackward>)


In [86]:
def edgeToMat(edges, node_num):
    A = torch.eye(node_num)
    for i in edges:
        A[i[0]][i[1]] += 1
        A[i[1]][i[0]] += 1
    D = torch.sum(A,dim=0)
    D = torch.diag(torch.pow(D , -0.5))
    return torch.matmul(torch.matmul(D,A),D)

In [87]:
a1= [[0,2],[0,3],[1,2],[2,3]]
print(edgeToMat(a1, 4))

tensor([[0.3333, 0.0000, 0.2887, 0.3333],
        [0.0000, 0.5000, 0.3536, 0.0000],
        [0.2887, 0.3536, 0.2500, 0.2887],
        [0.3333, 0.0000, 0.2887, 0.3333]])


In [88]:
#split dataset
train_mask = torch.zeros(node_num, dtype = torch.bool)
train_mask[:node_num - 1000] = 1               #1700左右training

val_mask = None                                
test_mask = torch.zeros(node_num, dtype = torch.bool)
test_mask[node_num - 500:] = 1                 # 500test

model = GCN(node_num, feat_dim, stat_dim, num_class).to(device)

#Adam是一种算法，可以百度了解
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay = 1e-3)
feat_Matrix = feat_Matrix.to(device)
matDAD = edgeToMat(cites, node_num).to(device)

for epoch in range(200):
    model.train()
    optimizer.zero_grad()

    out = model(matDAD, feat_Matrix)
    
    loss = F.nll_loss(out[train_mask], label_list[train_mask])
    _, pred = out.max(dim=1)
    
    correct = float(pred[train_mask].eq(label_list[train_mask]).sum().item())
    acc = correct / train_mask.sum().item()
    print('[Epoch {}/200] Loss {:.4f}, train acc {:.4f}'.format(epoch, loss.cpu().detach().data.item(), acc))
    
    loss.backward()
    optimizer.step()
    
    if (epoch + 1) % 10 == 0:
        model.eval()
        _, pred = model(matDAD, feat_Matrix).max(dim = 1)
        correct = float(pred[test_mask].eq(label_list[test_mask]).sum().item())
        acc = correct / test_mask.sum().item()
        print('Accuracy: {:.4f}'.format(acc))

[Epoch 0/200] Loss 1.9496, train acc 0.1247
[Epoch 1/200] Loss 1.9196, train acc 0.2061
[Epoch 2/200] Loss 1.8057, train acc 0.2482
[Epoch 3/200] Loss 1.6479, train acc 0.3519
[Epoch 4/200] Loss 1.6107, train acc 0.3841
[Epoch 5/200] Loss 1.4275, train acc 0.4063
[Epoch 6/200] Loss 1.3452, train acc 0.4093
[Epoch 7/200] Loss 1.2077, train acc 0.4660
[Epoch 8/200] Loss 1.1683, train acc 0.4766
[Epoch 9/200] Loss 1.1016, train acc 0.4977
Accuracy: 0.7580
[Epoch 10/200] Loss 1.0370, train acc 0.5246
[Epoch 11/200] Loss 1.0045, train acc 0.5293
[Epoch 12/200] Loss 0.9859, train acc 0.5263
[Epoch 13/200] Loss 0.9747, train acc 0.5281
[Epoch 14/200] Loss 0.8993, train acc 0.5527
[Epoch 15/200] Loss 0.8643, train acc 0.5632
[Epoch 16/200] Loss 0.8881, train acc 0.5433
[Epoch 17/200] Loss 0.8324, train acc 0.5550
[Epoch 18/200] Loss 0.8215, train acc 0.5708
[Epoch 19/200] Loss 0.7826, train acc 0.5925
Accuracy: 0.7920
[Epoch 20/200] Loss 0.8109, train acc 0.5726
[Epoch 21/200] Loss 0.7818, tra

[Epoch 177/200] Loss 0.6934, train acc 0.6112
[Epoch 178/200] Loss 0.7046, train acc 0.6007
[Epoch 179/200] Loss 0.6945, train acc 0.6095
Accuracy: 0.7580
[Epoch 180/200] Loss 0.7026, train acc 0.6095
[Epoch 181/200] Loss 0.7180, train acc 0.6019
[Epoch 182/200] Loss 0.6815, train acc 0.6224
[Epoch 183/200] Loss 0.7105, train acc 0.5989
[Epoch 184/200] Loss 0.7264, train acc 0.6030
[Epoch 185/200] Loss 0.7055, train acc 0.6007
[Epoch 186/200] Loss 0.7408, train acc 0.5878
[Epoch 187/200] Loss 0.7108, train acc 0.6107
[Epoch 188/200] Loss 0.6827, train acc 0.6194
[Epoch 189/200] Loss 0.7049, train acc 0.6165
Accuracy: 0.7540
[Epoch 190/200] Loss 0.7441, train acc 0.5872
[Epoch 191/200] Loss 0.7204, train acc 0.5984
[Epoch 192/200] Loss 0.7010, train acc 0.6030
[Epoch 193/200] Loss 0.7234, train acc 0.5913
[Epoch 194/200] Loss 0.7306, train acc 0.5931
[Epoch 195/200] Loss 0.6992, train acc 0.6247
[Epoch 196/200] Loss 0.6740, train acc 0.6358
[Epoch 197/200] Loss 0.7420, train acc 0.5919
